# **RAG model - NLP - házi feladat**
AX83OL

## Rövid ismertető

A következő colbertv2.0 ragatouille előtanított model a PTE valamint a PTE EHÖK hivatalos szabályzatain lett továbbtanítva.

Az alábbi weboldalak alatt lévő dokumentumokat használja:

*   A Pécsi Tudományegyetem Szervezeti és Működési Szabályzata és mellékletei:

    https://adminisztracio.pte.hu/adminisztracio/szabalyzatok_utasitasok/szervezeti_es_mukodesi_szabalyzat_es_mellekletek

*   PTE EHÖK Dokumentumtár:

    https://pteehok.hu/dokumentumtar/

Az összes dokumentum manuálisan le lett töltve a két oldalról, majd feltöltve ZIP fájlként a Google Drive-ra, onnan tud ez a notebook hozzáférni.

A továbbtanításhoz használni kívánt PDF-ek szerencsére szövegként tartalmaznak mindent, így azok felhasználhatóak voltak. (Illetve volt talán 2-3 melléklet ami nem, de azok amúgy sem túl kardinális témáról szólnak, így azokat kihagytam.)

Az összes használt doksi (=tanítóhalmaz) együttvéve kb. 1800 oldalt tesz ki.

# MEGVALÓSÍTÁS:

Feltelepítjük a két alapvetően szükséges modult (CUDA-hoz és RAG-hoz kell).

In [3]:
!pip install optimum auto-gptq

In [4]:
!pip install ragatouille

  Using cached faiss_cpu-1.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.0 MB)


Betöltjük az adott modelt.

In [5]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[Apr 09, 20:52:04] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Kell majd a PyPDF2 modul a PDF-ek feldolgozásához, így azt is feltelepítjük.

In [6]:
!pip install PyPDF2

Készítünk egy függvényt ami a Google Drive-ról letöltött és már kicsomagolt mappában lévő összes PDF doksin végigmegy, majd azokat betölti egy string változóba.

In [7]:
import os
from PyPDF2 import PdfReader

def read_pdfs(directory):
    """
    Read all PDF files in the specified directory and concatenate their contents into a single string.

    :param directory: str - Path to the directory containing PDF files.
    :return: str - Concatenated text content of all PDF files.
    """
    full_text = ""

    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)

        # Check if the file is a PDF
        if filepath.endswith(".pdf"):
            with open(filepath, "rb") as file:
                reader = PdfReader(file)

                # Iterate through each page of the PDF
                for page_num in range(len(reader.pages)):
                    page = reader.pages[page_num]
                    full_text += page.extract_text()

    return full_text

Letöltjük a forrásadatokat. (Nálam már le van, tehát nekem felesleges.)

In [8]:
!gdown 16Pn9bYZSY3no-QXvwKlsK0ORrdaG0TQX

Downloading...
From (original): https://drive.google.com/uc?id=16Pn9bYZSY3no-QXvwKlsK0ORrdaG0TQX
From (redirected): https://drive.google.com/uc?id=16Pn9bYZSY3no-QXvwKlsK0ORrdaG0TQX&confirm=t&uuid=698a26d8-885d-41f1-8270-ba63a03c9342
To: /content/Szabalyzatok.zip
100% 39.3M/39.3M [00:00<00:00, 128MB/s]


Kicsomagoljuk a letöltött ZIP-et, hogy alkalmazni tudjuk az így kapott mappára a read_pdfs függvényt.

In [9]:
import zipfile

# A ZIP fájl elérési útja
zip_file_path = "Szabalyzatok.zip"

# A célkönyvtár, ahova kicsomagoljuk a fájlokat
extract_to_directory = ""

# Megnyitjuk a ZIP fájlt és kicsomagoljuk a tartalmát
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

print("A ZIP fájl sikeresen kicsomagolva.")

A ZIP fájl sikeresen kicsomagolva.


Alkalmazzuk az előbb említett függvényt.

In [10]:
# Directory containing the PDF files
pdf_directory = "Szabalyzatok"

# Read all PDF files in the directory and concatenate their contents
full_document = read_pdfs(pdf_directory)

# Check the length of the concatenated text
print(len(full_document))
type(full_document)

5397394


str

In [11]:
full_document

Output hidden; open in https://colab.research.google.com to view.

A tanítóhalmaz indexeléséhez eltávolítjuk a ragatoullie modullal automatikusan installált CPU modult, majd meggyőződünk, hogy a GPU-s verzió fent van-e.

In [12]:
!pip uninstall faiss-cpu
!pip install faiss-gpu

Found existing installation: faiss-cpu 1.8.0
Uninstalling faiss-cpu-1.8.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/faiss/*
    /usr/local/lib/python3.10/dist-packages/faiss_cpu-1.8.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/faiss_cpu.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.10/dist-packages/faiss_cpu.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.10/dist-packages/faiss_cpu.libs/libquadmath-96973f99.so.0.0.0
Proceed (Y/n)? Y
  Successfully uninstalled faiss-cpu-1.8.0


### Indexeljük a tanítóhalmazt, itt hossznak 180-at adtam meg: ez azt jelenti, hogy milyen hosszú részekre darabolja fel a tanítóhalmazt. => Majd a válaszok milyen hosszúak legyenek max.

In [ ]:
RAG.index(collection=[full_document], index_name="PTE", max_document_length=180, split_documents=True)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Apr 09, 20:56:18] #> Creating directory .ragatouille/colbert/indexes/PTE 


[Apr 09, 20:56:20] [0] 		 #> Encoding 18190 passages..


  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
 88%|████████▊ | 44/50 [13:13<01:44, 17.37s/it]

A következő függvénnyel tudunk kérdéseket feltenni a megtanult doksikkal kapcsolatban.

In [ ]:
def inputFunction(question):
  k = 3 # How many documents you want to retrieve, defaults to 10, we set it to 3 here for readability
  results = RAG.search(query=question, k=k)
  return results[0]['content'].replace("\n","").replace("   ","  ").replace("  "," ")

Tesztként:

In [ ]:
result = inputFunction("Mikor utasítja el érdemi vizsgálat nékül az EHÖK a kollégiumi jelentkezésemet?")
print(result)

## Betöltünk a Hugging Face-ről egy transformers szöveggeneráló modelt amely támogat magyar nyelvet (és remélhetőleg elbírja a Colab rendszere is). Majd átadjuk ennek az előző colbertv2.0 RAG model válaszát.

In [ ]:
from transformers import pipeline

def theFunction(theQuestion):

  answerSkeleton = inputFunction(theQuestion)

  prompt = f"{theQuestion} válaszold meg a következő szöveg alapján, csak a szöveget használhatod fel: {answerSkeleton}"
  generator = pipeline(task="text-generation", model="NYTK/PULI-GPT-2", max_length=300)

  return generator(prompt)[0]["generated_text"]

In [ ]:
theFunction("Mikor utasítja el érdemi vizsgálat nékül az EHÖK a kollégiumi jelentkezésemet?")

A Gradio-t is fel kell telepítenünk a webes GUI miatt.

In [ ]:
!pip install gradio

Megvalósítunk egy egyszerű webes GUI-t:

In [ ]:
import gradio as gr

# Létrehozunk egy Gradio interfészt a függvényünkkel
interface = gr.Interface(
    fn=theFunction,
    inputs="text",
    outputs="text",
    title="RAG Model Summary Generator (PTE admin edition)",
    description="Generates a summary for a given input text using the RAG model. This RAG model trained on many administrative documents of PTE and PTE EHÖK.",
)

# A Gradio interfész indítása
interface.launch()